# Data upload

In [1]:
import tarfile
import pandas as pd
import numpy as np
import re
import os
import gzip

directory = os.getcwd()
data_path = os.path.join(directory, 'Data')

In [ ]:
# List the .tar files in the source directory and extract them in the corresponding subfolder
tar_files = [file for file in os.listdir(data_path) if file.endswith('.gz')]

for tar_file in tar_files:
    tar_file_path = os.path.join(data_path, tar_file)

    extraction_folder_name = os.path.basename(tar_file_path).split('.', 1)[0]
    folder_path = os.path.join(data_path, extraction_folder_name)
    os.makedirs(folder_path, exist_ok=True)

    with tarfile.open(tar_file_path, 'r') as tar:
        tar.extractall(path=folder_path)

In [2]:
def text_to_csv(path, filename):
    '''
        This function takes as input the path where the .txt file is located as well as the filename and will create a .tsv file 
        in the same folder from the data in the .txt file. 
        
        The .txt file is read line by line and a dictionary is created to which the different values for each key are appended as 
        the file is read. The "nan" are replaced by empty space ('') to be better recognized when creating a dataframe.
    '''
    main_path = os.path.join(path, filename)
    if not os.path.exists(main_path + '.tsv'):
        with gzip.open(main_path + '.txt.gz', 'rt', encoding="utf8") as file_txt:
            with open(main_path + '.tsv', 'w', encoding="utf8") as file_tsv:
                first = True
                obj = {}
                for line in file_txt:
                    if line == '\n':
                        if first:
                            file_tsv.write("\t".join(obj.keys()) + "\n")
                            first=False
                        file_tsv.write("\t".join(obj.values()) + "\n")
                        obj = {}
                        continue

                    line = re.sub(r'\bnan\b', '', line)
                    key, value = line.strip().split(":", 1)
                    obj[key] = value
                
                if obj: file_tsv.write("\t".join(obj.values()) + "\n")

In [3]:
# Definining the path for the files
RB_path = os.path.join(data_path, 'RateBeer')
BA_path = os.path.join(data_path, 'BeerAdvocate')
MB_path = os.path.join(data_path, 'matched_beer_data')

In [ ]:
#Transformation of the .txt files
text_to_csv(RB_path, 'ratings')
text_to_csv(RB_path, 'reviews')

text_to_csv(BA_path, 'ratings')
text_to_csv(BA_path, 'reviews')

text_to_csv(MB_path, 'ratings_ba')
text_to_csv(MB_path, 'ratings_with_text_ba')
text_to_csv(MB_path, 'ratings_rb')
text_to_csv(MB_path, 'ratings_with_text_rb')

In [2]:
# Assigning variables
RB_beers = pd.read_csv(RB_path + '/beers.csv')
RB_breweries = pd.read_csv(RB_path + '/breweries.csv')
RB_users = pd.read_csv(RB_path + '/users.csv')
RB_ratings = pd.read_csv(RB_path + '/ratings.tsv', sep='\t')
RB_reviews = pd.read_csv(RB_path + '/reviews.tsv', sep='\t')

BA_beers = pd.read_csv(BA_path + '/beers.csv')
BA_breweries = pd.read_csv(BA_path + '/breweries.csv')
BA_users = pd.read_csv(BA_path + '/users.csv')
BA_ratings = pd.read_csv(BA_path + '/ratings.tsv', sep='\t')
BA_reviews = pd.read_csv(BA_path + '/reviews.tsv', sep='\t')

MB_beers = pd.read_csv(MB_path + '/beers.csv', header=1)
MB_breweries = pd.read_csv(MB_path + '/breweries.csv', header=1)
MB_users = pd.read_csv(MB_path + '/users.csv', header=1)
MB_users_approx = pd.read_csv(MB_path + '/users_approx.csv', header=1)
MB_ratings = pd.read_csv(MB_path + '/ratings.csv', header=1)
MB_ratingsBA = pd.read_csv(MB_path + '/ratings_ba.tsv', sep='\t')
MB_ratingsBA_txt = pd.read_csv(MB_path + '/ratings_with_text_ba.tsv', sep='\t')
MB_ratingsRB = pd.read_csv(MB_path + '/ratings_rb.tsv', sep='\t')
MB_ratingsRB_txt = pd.read_csv(MB_path + '/ratings_with_text_rb.tsv', sep='\t')

In [ ]:
beers.nbr_matched_valid_ratings.unique()

In [ ]:
display(beers)
display(breweries)
display(users)

In [ ]:
users['nbr_ratings'].sort_values(ascending=False).plot(kind='hist', range = (20,1000), bins= 100)
# average rating/ overall rating in time